# Assignment 4:
    test if the distribution of ages is the same for 
    
    1) male and female riders
    
    2) day and night riders
    
    use 3 tests: KS, Pearson's, Spearman's. 
    
    For the KS do the test with the entire dataset and with a subset 200 times smaller
    
    Choose a significant threshold for the whole eercise. 
    
    For each test phrase the Null Hypothesis in words.
    
    Describe the return of the scipy function you use in each case.
    
    State the result in terms of rejection of the Null.

In [ ]:
# my usual imports and setups
import pylab as pl
import pandas as pd
import numpy as np
#imports downloader
from getCitiBikeCSV import getCitiBikeCSV

%pylab inline

import os
#this makes my plots pretty! but it is totally not mandatory to do it
import json
#s = json.load( open(os.getenv ('PUI2016')+"/fbb_matplotlibrc.json") )
#pl.rcParams.update(s)



In [ ]:
#efining a significant threshold for the whole exercise
alpha = 0.05

# I am reading in data from January 2015. It would be a good idea to use data from warmer months, since there are more riders in the warm weather and the more data the smaller the STATISTICAL NOISE. You should use data from multiple months, joining multiple datasets, thus addressing systematic errors as well

In [ ]:
datestring = '201506' #Using data set for month of June
getCitiBikeCSV(datestring)

In [ ]:
df = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + '-citibike-tripdata.csv')
df.head()

In [ ]:
# df is the dataframe where the content of the csv file is stored
df['date'] = pd.to_datetime(df['starttime'])
# note that with dataframes I can refer to variables as dictionary keys, 
# i.e. df['starttime'] or as attributes: df.starttime. 
df.head()

In [ ]:
df.columns

# BY GENDER
splitting data by gender and extracting the age happens in the cell below

In [ ]:
#df is the dataframe where the content of the csv file is stored
df['ageM'] = 2015 - df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 1)]
df['ageF'] = 2015 - df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 2)]

In [ ]:
#dropping some data I no longer need
#... your code here...
df = df[['tripduration', 'gender', 'date', 'ageM', 'ageF']]
df.head()

In [ ]:
#your df should look something like this
# look at these data carefully... you may see someinteresting values!
df.describe()

In [ ]:
# dropping NaN values
df['ageM'].dropna(inplace= True)
df['ageF'].dropna(inplace= True)
df.describe()

lets split age in 10 year bins. the dataset is very large, so i could be split in smaller bins, but I will chose 10 years in the interest of time. 
the bin size choice should be a balance between properly sample the age space, have enough counts in each bin that the statistical noise is not significant (remember that is > sqrt(N)!) and the computational requirement to computatinal facilities ratio. 

the next several steps are needed if you want to code up the KS test from scratch. that is for extra credit, so if you do not want to do it you may not need to plot split the distribution in bins and create the cumulative HOWEVER it is a great idea to do it anyways to explore your data viaually! remember Ascombe's quartet!!

In [ ]:
# plotting histogramswith pandas is a bitdifferent than with pylab
# this is a VERY USEFUL syntaxfor you to knonw!
bins = np.arange(10, 99, 5)
axM = df.ageM.groupby(pd.cut(df.ageM, bins)).agg([count_nonzero]).plot(kind='bar', 
                                                                legend=False)
axM.set_title("male riders")
axF = df.ageF.groupby(pd.cut(df.ageF, bins)).agg([count_nonzero]).plot(kind='bar',
                                                                legend=False)
axF.set_title("female riders")


# Figure 1: histogrammed distribution of riders' ages by gender

this is how the cumulative distributions look like.  Notice that i am normalizing them! if i want to reat an observed distribution like a probablility distribution i have to normalize it

In [ ]:
#print df.ageS, df.ageS.cumsum()

csM=df.ageM.groupby(pd.cut(df.ageM, bins)).agg([count_nonzero]).cumsum()

csF=df.ageF.groupby(pd.cut(df.ageF, bins)).agg([count_nonzero]).cumsum()

print (np.abs(csM / csM.max()-csF / csF.max()))

pl.plot(bins[:-1] + 5, csM / csM.max(), label = "M")
pl.plot(bins[:-1] + 5, csF / csF.max(), label = "F")
pl.plot(bins[:-1] + 5, np.sqrt(csF / csF.max() - csM / csM.max())**2, 'k-',
        label = "difference")
pl.xlabel("Age")
pl.ylabel("Normalized Cumulative Number")
pl.legend()

# Figure 2: the cumulative distribution of CitiBike riders' ages by gender

They look similar! But the difference gets to 10%. If I wanted to code the KS test by hand I woud have everything I need: the normalized cumulative distributions can be subtracted from each other and the max distance can calculated. 

Notice that there may be NaN values you are gonna have to deal with! 
You can do that for example with a Boolean statementsuch as  df.ageF[~np.isnan(df.ageF)] or you can use numpy functions that deal with Nan values: nansum, nanmean, nanstd...

lets run the scipy KS test

In [ ]:
import scipy.stats

# KS tests to compare 2 samples

http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ks_2samp.html


# NULL HYPOTHESIS: In the cumulative distribution of CitiBike riders' ages by gender, there is no difference between age of men and women, significance level p=0.05

# $H_0: P_0 - P_1 =$    0
    
# $H_a: P_0 - P_1 \neq$ 0   

In [ ]:
ks = scipy.stats.ks_2samp(df.ageM, df.ageF)
print (ks)

# FILL IN THE CELL BELOW!

In [ ]:
## your words here!...
## this cell is for you to tell me what the scipy KS test returned and what it means in terms of NULL HYPOTHESIS
## to do that refer to the scipy documentation to understand the output of the scipy.stats.ks_2samp function
print ('KS Test statistic is ', ks[0])
print ('p-value for the test is ', ks[1])
print ('Since the p-value is lower than our significance level, we can infer that the NULL hypothesis is rejected')




The scipy.stats KS test already tells me the significance and the p-value. 

The next few cells are here just to show you how you would obtain the same result by hand, but they are **not required**. 

Remember: the Null hypothesis is rejected if 

$D_KS(n1,n2) > c(\alpha) \sqrt{\frac{(n1 + n2)}{n1n2}}$

(see class notes) where $c(\alpha$) is the inverse of the KS distribution, and you do not have to know how to get that cause there are tables that list critical values!! 

http://www.real-statistics.com/tests-normality-and-symmetry/statistical-tests-normality-symmetry/kolmogorov-smirnov-test/kolmogorov-distribution/

But also this result depends in your choice of binning through, and thustheresultyou get by hand may not be exactly the same as the one the KS returns. Either way: this is how you would calculate the KS statistics by hand.

In [ ]:
#here is the critical values tablel. Have you chosen your significance level yet?? you should do it first thing!
#from IPython.display import Image
#Image(filename="../plotsforclasses/ks2sample_table.png")

# Now redo the test with a subsample of the data: take only 1 ride every of 200 

In [ ]:
## your words here!...
## this cell is for you to redo the test with reducted dataset 
## and tell me what the scipy KS test returned and what it means in terms of NULL HYPOTHESIS


kss = scipy.stats.ks_2samp(df.ageM.sample(frac=0.005, random_state =15), df.ageF.sample(frac=0.005, random_state =15)) 
#using df.sample to select a sample of 1/200 size of the data set
print (kss)


In [ ]:
print ('KS Test statistic is ', kss[0])
print ('p-value for the test is ', kss[1])
print ('Since the p-value the sample is lower than our significance level, we can infer that the NULL hypothesis is rejected')

# Pearson's  test for correlation

## notice that the Pearson's is a pairwise test: the samples need to be 
## a. sorted and 
## b. the same size!

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr



# NULL HYPOTHESIS: In the cumulative distribution of CitiBike riders' ages by gender, there is no difference between age of men and women, significance level p=0.05

# $H_0: P_0 - P_1 =$    0
    
# $H_a: P_0 - P_1 \neq$ 0 

In [ ]:
# your code here!
# wrangle the data as needed
# please perform the Pearson's test 
# and tell me what you find in terms of NULL hypothesis
ageMr = df.ageM.sample(len(df.ageF))
ageMsort = ageMr.sort_values(ascending=True)
ageFsort = df.ageF.sort_values(ascending=True)

pt = scipy.stats.pearsonr(ageMsort, ageFsort) 

print ('Pearson Test statistic is ', pt[0])
print ('p-value for the test is ', pt[1])
print ('Since the p-value the sample is lower than our significance level, we can infer that the NULL hypothesis is rejected. Pearson Test statistic number is close to 1, which means the data sets are highly correlated.')

# Spearman's  test for correlation

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy.stats.spearmanr

# NULL HYPOTHESIS: In the cumulative distribution of CitiBike riders' ages by gender, there is no difference between age of men and women, significance level p=0.05

# $H_0: P_0 - P_1 =$    0
    
# $H_a: P_0 - P_1 \neq$ 0 

In [ ]:
# your code here!
# wrangle the data as needed
# please perform the Spearman's test and tell me what you find in terms of NULL hypothesis
st = scipy.stats.spearmanr(ageMsort, ageFsort)
print ('Spearman Test statistic is ', st[0])
print ('p-value for the test is ', st[1])
print ('Since the p-value the sample is lower than our significance level, we can infer that the NULL hypothesis is rejected. Spearman Test statistic number is close to 1, which means the data sets are highly correlated but not the same.')

## please comment on the numbers that you get in the light of the scipy manual: what is rerutned? what does it mean??

## EXTRA CREDIT:  CHOOSE ONE OF THE TESTS, AND TEST THE CORRELATION OF AGES  OF RIDERS DURING THE DAY VS DURING THE NIGHT!



In [ ]:
df.head()

In [134]:
df.head()

,tripduration,gender,ageM,ageF
date,,,,
2015-06-01 00:00:00,1338,1,31.0,NaN
2015-06-01 00:00:00,290,1,18.0,NaN
2015-06-01 00:01:00,634,1,22.0,NaN
2015-06-01 00:01:00,159,1,34.0,NaN
2015-06-01 00:02:00,1233,0,NaN,NaN


In [133]:
#df is the dataframe where the content of the csv file is stored
# Assuming Day is 6AM - 6PM and Night 6PM - 6AM

#df.set_index('date', inplace=True)
df['age_day'] = df.between_time(start_time='6:01', end_time='18:00')
df['age_night'] = df.between_time(start_time= '18:01', end_time= '6:00')

KeyboardInterrupt: 

In [124]:
index = pd.DatetimeIndex(df.date)
df.iloc[index.indexer_between_time('6:00','18:00')]

df.head()

AttributeError: 'DataFrame' object has no attribute 'date'